# FEM-koe
## Tehtävä 1
Teräsputki, jonka ulkohalkaisija on $20\,\mathrm{mm}$ ja
sisähalkaisija $15\,\mathrm{mm}$, on päistään kiinnitetty.
Laske elementtimenetelmällä pisteen $B$ siirtymä ja
tukireaktiot putken päissä. $E = 210\,\mathrm{GPa}$.

In [1]:
using DataFrames
using Unitful
using SymPy

D = 20e-3
d = 15e-3

A = pi/4 * (D^2 - d^2)
E = 210e9
L = [0.3, 0.7]
P = 16e3

bar(L) = A*E/L * [1 -1; -1 1]

elements = [bar(L[1]), bar(L[2])]

K = zeros(3, 3)
F = zeros(3)
u = zeros(3)

F[2] = -P

K[1:2, 1:2] += elements[1]
K[2:3, 2:3] += elements[2]

u[2] = F[2] / K[2, 2];
F = K*u;

In [2]:
DataFrame(u=u*u"m" .|> u"mm", 
          F=F*u"N" .|> u"kN")

,u,F
,Quantit…,Quantit…
1,0.0 mm,11.2 kN
2,-0.11641 mm,-16.0 kN
3,0.0 mm,4.8 kN


## Tehtävä 2
Kannatin on mallinnettu yhdellä sauvaelementillä
(1) ja yhdellä palkkielementillä (2). Olet laskenut rakenteen tuntemattomat solmusiirtymät (SI-yksiköinä):
$$
\left\{
    \begin{array}{lll}
        u_2 \\
        v_2 \\
        \theta_2
    \end{array}
\right\} = \left\{
    \begin{array}{ccc}
        -0.00634 \\
        -0.048 \\
        -0.024
    \end{array}
\right\}
$$
Määritä sauvan (elementin 1) normaalivoima matriiseja käyttäen. Sauva on terästä $E = 210\,\mathrm{GPa}$, $A = 1.0 \cdot 10^{-3}\,\mathrm{m}^2$.

In [3]:
θ = -atan(1.5/3)
A = 1e-3
L = sqrt(3^2 + 1.5^2)
u = [0, 0, -0.00634, -0.048]

C, S = cos(θ), sin(θ)

u_ = [C S 0 0; 0 0 C S] * u

k = A*E / L * [1 -1; -1 1]
F_ = k * u_
F_*u"N" .|> u"kN"

2-element Array{Quantity{Float64,𝐋 𝐌 𝐓^-2,Unitful.FreeUnits{(kN,),𝐋 𝐌 𝐓^-2,nothing}},1}:
 -988.96 kN
  988.96 kN

## Tehtävä 3
Sauva (pituus $L$, taivutusjäykkyys $EI$) on
nivelellisesti tuettu kummastakin päästä. Laske
nurjahdusvoima elementtimenetelmällä käyttäen
yhtä palkkielementtiä.

In [12]:
@vars E I L P λ

K_t = E*I/L^3*[ 12   6L -12   6L;
                6L 4L^2 -6L 2L^2;
               -12  -6L  12  -6L;
                6L 2L^2 -6L 4L^2]

K_g = P / 30L * [36   3L -36   3L;
                 3L 4L^2 -3L -L^2;
                -36  -3L  36  -3L;
                 3L -L^2 -3L 4L^2]

A = (K_t - λ*K_g)[[2, 4], [2, 4]]

2×2 Array{Sym,2}:
 4*E*I/L - 2*L*P*λ/15    2*E*I/L + L*P*λ/30
   2*E*I/L + L*P*λ/30  4*E*I/L - 2*L*P*λ/15

In [13]:
using LinearAlgebra: det
solve(det(A), λ*P)[1]

12*E*I
------
   2  
  L   

## Tehtävä 4
Laske elementtimenetelmällä oheisen palkin
taipumat ja kiertymät solmupisteissä. Ratkaise
siirtymät Gaussin eliminaatiomenetelmällä, jonka
välivaiheet on esitettävä.  
$P = 12\,\mathrm{kN}$, $L = 2\,\mathrm{m}$, $E = 210\,\mathrm{GPa}$, $I = 2\cdot10^{-5}\,\mathrm{m}^4$.

In [37]:
P = 12e3
L = 2
I = 2e-5
E = 210e9

beam = E*I/L^3*[ 12   6L -12   6L;
                 6L 4L^2 -6L 2L^2; 
                -12  -6L  12  -6L;
                 6L 2L^2 -6L 4L^2]
K = zeros(6, 6)
F = [0, 0, 0, 0, -P, 0]

K[1:4, 1:4] += beam
K[3:6, 3:6] += beam
A = K[4:6, 4:6]
b = F[4:6]
rationalize.([A b] / 2100000)

3×4 Array{Rational{Int64},2}:
  8//1  -3//1   2//1   0//1
 -3//1   3//1  -3//1  -1//175
  2//1  -3//1   4//1   0//1

In [17]:
rationalize.(A \ b)

3-element Array{Rational{Int64},1}:
 -1//350
 -1//75
 -3//350

In [38]:
A \ b

3-element Array{Float64,1}:
 -0.002857142857142857
 -0.013333333333333334
 -0.008571428571428572